In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
import gc

# Classifier Libraries
import collections
from scipy.stats import randint, uniform
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm_notebook,trange, tqdm

# Other Libraries
# from imblearn.datasets import fetch_datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
# from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

import mlrose

In [26]:
df = pd.read_csv("../data/creditcardfraud/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [27]:
std_scaler = StandardScaler()
rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)

In [28]:

X = df.drop('Class', axis=1)
y = df['Class']

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels


# Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

df.head()

Train: [265518 180305  42664 ...  29062  13766  17677] Test: [263020  11378 147283 ... 274532 269819  64170]
Train: [ 72227 114282  16818 ... 264471 191914 284017] Test: [202638  32978 128121 ... 244024 127667  48318]
Train: [ 20895 114622 167683 ... 244502 178972 218506] Test: [284352  82483  90981 ... 171224 168807 271602]
Train: [122248 181660 194400 ... 104631 277586  29432] Test: [225673  63348  68025 ... 279451  77554  76043]
Train: [241684 223467 136928 ...  86495 160550  49633] Test: [157557 204860  83760 ... 251478 178967 216850]


,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [29]:
# Since our classes are highly skewed we should make them equivalent in order to have a normal distribution of the classes.

# Lets shuffle the data before creating the subsamples

df = df.sample(frac=1)

# amount of fraud classes 492 rows.
fraud_df = df.loc[df['Class'] == 1]
non_fraud_df = df.loc[df['Class'] == 0][:492]

normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

new_df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
220860,2.607699,0.677111,-0.679435,0.618923,1.194657,-0.425657,-2.250719,0.751678,-0.570801,-3.343568,...,-1.206486,2.325145,-2.408189,0.156442,0.292784,1.115726,-0.234767,0.407610,0.109090,0
30473,-0.105359,-0.572728,-4.194074,4.382897,-5.118363,4.455230,-4.812621,-1.224645,-7.281328,3.332250,...,0.579098,1.550473,0.614573,0.028521,0.013704,-0.149512,-0.131687,0.473934,0.473757,1
49671,-0.167819,-0.476216,1.149887,-0.217033,1.232729,1.369140,-1.211925,-0.265780,-0.731111,0.234122,...,-0.296327,-0.091480,-0.181917,0.024644,0.308051,0.328784,-0.399886,0.055795,0.032807,0
6882,-0.292042,-0.891505,-4.617217,1.695694,-3.114372,4.328199,-1.873257,-0.989908,-4.577265,0.472216,...,-0.039046,0.481830,0.146023,0.117039,-0.217565,-0.138776,-0.424453,-1.002041,0.890780,1
142557,4.643750,0.001140,-1.430864,-0.802529,1.123320,0.389760,-0.281214,-0.055123,1.326232,0.195700,...,0.634184,0.325575,0.014002,0.844946,0.114963,0.156365,-0.619437,-0.120351,0.035594,1


In [30]:
X = new_df.drop('Class', axis=1)
y = new_df['Class']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

## Gradient Descent Baseline

In [39]:
cross_val_score(mlrose.NeuralNetworkClassifier(hidden_nodes = [30], activation ='relu', 
                    algorithm ='gradient_descent', 
                    max_iters = 1000,
                    learning_rate = 0.0001, 
                    curve=True), X_train, y_train, cv=5)

array([0.94303797, 0.89873418, 0.94904459, 0.94904459, 0.93630573])

In [35]:
param_dist_rhc = {
    "max_attempts": randint(1, 501),
    "learning_rate": uniform(.01, 1),
    "restarts": randint(1, 101)       
}

rand_nn_rhc = RandomizedSearchCV(mlrose.NeuralNetworkClassifier(hidden_nodes = [30], activation ='relu', 
                             algorithm ='random_hill_climb', max_iters=2000, curve=True),
                             param_dist_rhc, cv=5, n_jobs=-1, n_iter=50)

In [ ]:
rand_nn_rhc.fit(X_train, y_train)
rand_nn_rhc_clf = rand_nn_rhc.best_estimator_
rand_nn_rhc_score = cross_val_score(nn_rhc_clf, X_train, y_train, cv=5)

In [50]:
param_dist_ga = {
    "max_attempts": randint(1, 501),
    "pop_size": randint(1, 201),
    "mutation_prob": uniform(0, 1)       
}

rand_nn_ga = RandomizedSearchCV(mlrose.NeuralNetworkClassifier(hidden_nodes = [30], activation ='relu', 
                             algorithm ='genetic_alg', max_iters=2000, curve=True),
                             param_dist_ga, cv=5, n_jobs=-1, n_iter=50)

In [51]:
rand_nn_ga.fit(X_train, y_train)
rand_nn_ga_clf = rand_nn_ga.best_estimator_
rand_nn_ga_score = cross_val_score(rand_nn_ga_clf, X_train, y_train, cv=5)

In [60]:
nn_ga_clf

NeuralNetworkClassifier(activation='relu', algorithm='genetic_alg', bias=True,
            clip_max=10000000000.0, curve=True, early_stopping=False,
            hidden_nodes=[30], learning_rate=0.1, max_attempts=10,
            max_iters=371, mutation_prob=0.8397542964732706, pop_size=53,
            restarts=0,
            schedule=<mlrose.decay.GeomDecay object at 0x000002B88F4337F0>)

In [78]:
param_dist_sa = {
    "schedule": [mlrose.ExpDecay(), mlrose.GeomDecay(), mlrose.ArithDecay()],
    "max_attempts": randint(1, 501)     
}

rand_nn_sa = RandomizedSearchCV(mlrose.NeuralNetworkClassifier(hidden_nodes = [30], activation ='relu', 
                             algorithm ='simulated_annealing', max_iters=2000, curve=True),
                             param_dist_sa, cv=5, n_jobs=-1, n_iter=50)

In [79]:
rand_nn_sa.fit(X_train, y_train)
rand_nn_sa_clf = rand_nn_sa.best_estimator_
rand_nn_sa_score = cross_val_score(rand_nn_sa_clf, X_train, y_train, cv=5)

In [80]:
rand_nn_sa.cv_results_['params']

[{'max_attempts': 352, 'schedule': <mlrose.decay.GeomDecay at 0x2b88ab8f940>},
 {'max_attempts': 35, 'schedule': <mlrose.decay.ArithDecay at 0x2b88ab8f780>}]

In [82]:
rand_nn_sa_score

array([0.78481013, 0.81012658, 0.85350318, 0.7133758 , 0.84076433])